In [1]:
%load_ext autoreload
%autoreload 2

# Efficient Multi-Party Private Set Intersection

### By Malia Kency and John Owens

First, load imports

In [2]:
import math
import protocol as P
import bloom_filter as BF
import pandas as PD

Set constant protocol parameters:

   * $NumPlayers$ = Total number of players $P_i$
   * $PlayerInputSize$ = Size of the player's input sets
   * $N_{BF}$ = Length of the Bloom Filter
   * $k$ = Number of hash functions to use
   * $\kappa = SecParam$ = Security paremeter. 40 in the paper's example
   * $N_{Maxones}$ = Maximum number of ones a player is allowed after cut-and-choose
   * $p$ = percentage of total messages to be used for cut-and-choose
   * $a$ = sampling weight of 1s vs. 0s for every $P_i$

Then calculate the rest:
   * $N_{OT}$ = Total number of Random Oblivious Transfers
   * $m_h^1$ = The number of 1's a player chooses
   * $\gamma = gamma$ = Verifies the correct relationship between $p, \kappa, m_h^1$
   * $\gamma^* = gammaStar$ = Verifies the correct relationship between $p, \kappa, N_{OT}$
   * $\varphi$
   * $\pi$
   * $\epsilon$
   * $\lambda$

   

In [3]:
NumPlayers = 3 
PlayerInputSize = 5
Nbf = 10
k = BF.GetNumHashFuncs(Nbf, PlayerInputSize)
SecParam = 20
Nmaxones = 10
p = 0.25
a = 0.3

# Initialize the protocol with the parameters above
Protocol = P.Protocol(Nmaxones, PlayerInputSize, k, p, a, SecParam, Nbf)

# Add the players to the protocol
Protocol.AddPlayers(NumPlayers)

Player 0 created
Player 1 created
Player 2 created


After parameters are set, we can simluate the first major step of the protocol: Random OT

Note: The number of Oblivious Transfers that occur is determined by $N_{OT}$, which is the result of several calculations in `security_parameters.py`

In [16]:
Protocol.Perform_RandomOT(Protocol.Params.a, Protocol.Params.Not)
Protocol.Report_PlayerMessages()

Performing 67 Random Oblivious Transfers...
                                     P0's 1s  \
1     79385521461839476913602713971809239000   
2     95843188183947512284759992019121661321   
3    321417030532038087619867779859679378427   
4    247564699884232018954608966577406111889   
5    181769869763132698203040545802833368257   
...                                      ...   
134  314471991917618508102007260714213214091   

                                     P0's 0s  \
1     93871778688780312720321773628345112618   
2    311171856673962675149118537734215301921   
3    277152525336698952202404909683310143623   
4    182989987425593319525406455842791314601   
5    151992231802778606202965544118060859882   
...                                      ...   
134  263281102740236273316967273015506758021   

                                     P1's 1s  \
1                                          0   
2                                          0   
3                                         